In [4]:
# standard python tools
import numpy as np
from glob import glob
import os

In [10]:
# make sure pyqae is available
import pyqae as pq
    
# check to see if it is running on pyspark (if not use the simplespark backend)
try:
    import pyspark
except ImportError:
    print('Using a fake spark backend for processing')

    import pyqae.simplespark as pyspark
    sc = pyspark.LocalSparkContext()

from pyqae.images import viz
# lazy image 
from pyqae.images.lazy import paths_to_tiled_image, DiskMappedLazyImage, backends
print(backends)

Using a fake spark backend for processing
[<class 'pyqae.images.lazy.LazyImagePillowBackend'>]


In [19]:
base_path = os.path.join("..","test","resources")
little_image_list = glob(os.path.join(base_path,"multilayer_tif","*.tif"))
big_image_list = [little_image_list[-1]]

In [20]:
start_img = DiskMappedLazyImage(big_image_list[0], backends[0])
print(start_img.shape)

(120, 120)


In [21]:
def create_line_break(x_tile_size, x_dim):
    return [(x, min(x+x_tile_size, x_dim)) for x in range(0, x_dim, x_tile_size)]

def create_tiles(x_tile_size, x_dim, y_tile_size, y_dim):
    return [(x_tile, y_tile) for x_tile in create_line_break(x_tile_size, x_dim) 
            for y_tile in create_line_break(y_tile_size, y_dim)]
def tile_dict(tile_list):
    return [{'x_start': int(xs), 'y_start': int(ys), 
             'x_end': int(xe), 'y_end': int(ye)} for ((xs, xe), (ys, ye)) in tile_list]

In [22]:
print(create_line_break(256, 1025))

[(0, 256), (256, 512), (512, 768), (768, 1024), (1024, 1025)]


In [23]:
im_size = np.random.randint(1000, 4000, size = 2)
for tile_size in np.random.randint(100, 300, size = (3,2)):
    c_tiles = create_tiles(tile_size[0], im_size[0],tile_size[1], im_size[1])
    tile_sum = np.sum([(xe-xs)*(ye-ys) for ((xs, xe), (ys, ye)) in c_tiles])
    print(np.random.permutation(c_tiles)[0:2])
    assert tile_sum == im_size[0]*im_size[1], "Total pixel count cannot change, {} != {}".format(im_size[0]*im_size[1], tile_sum)

[[[1507 1644]
  [ 388  582]]

 [[2603 2740]
  [ 776  970]]]
[[[2223 2470]
  [ 931 1064]]

 [[ 741  988]
  [ 133  266]]]
[[[1300 1560]
  [ 141  282]]

 [[1560 1820]
  [ 141  282]]]


In [24]:
sqlContext.createDataFrame(tile_dict(c_tiles))

NameError: name 'sqlContext' is not defined